In [ ]:
import sys
sys.path.append('../..')

import pickle

from ml.dataset.corpus import *
from ml.dataset.base_predictor import *
import numpy as np

import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.models import load_model

from keras.layers import LSTM, Embedding
from keras.optimizers import RMSprop

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.engine.topology import Input

from sklearn.metrics import accuracy_score
from keras.layers import concatenate
from keras.layers.merge import Concatenate

from keras.utils import to_categorical

from matplotlib import pyplot as plt
from IPython.display import clear_output
DATASET_HOME='../../datasets/'

In [ ]:
with open(DATASET_HOME+'simple/simple_dataset_ClassifyChordsMapper.pickle', 'rb') as f:
    songs = pickle.load(f)

In [ ]:
model_chords_input = Input((334, ), batch_shape=(1,1))
model_chords_embedding = Embedding(794, 20, input_length=1, batch_input_shape=(1,1))(model_chords_input)
model_notes_input = Input((334, ), batch_shape=(1,1))
model_notes_embedding = Embedding(128, 20, input_length=1, batch_input_shape=(1,1))(model_notes_input)

In [ ]:
merged = keras.layers.concatenate([model_chords_embedding, model_notes_embedding])

lstm = LSTM(20, stateful=True)(merged)
model_dense = Dense(794, activation='softmax')(lstm)

merged_model = keras.Model(inputs=[model_chords_input, model_notes_input], outputs=model_dense)
merged_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(allX, allY)

In [ ]:
all_len = 0
acc = 0
for i, (x, y) in enumerate(zip(X_train, y_train)):
    print(i)
    merged_model.fit(x, y, epochs=1, batch_size=1, verbose=1)
    acc += merged_model.evaluate(x, y, batch_size=1)[1] * len(y)
    all_len += len(y)
    print("current_accuracy: ", acc / all_len)
    merged_model.reset_states()

In [ ]:
merged_model.save(DATASET_HOME+"simple/LSTM.h5")

In [ ]:
merged_model = keras.models.load_model(DATASET_HOME+"simple/LSTM.h5")

In [ ]:
corpus = SongCorpus()
corpus.load_from_file(DATASET_HOME+'simple/simple_dataset_SimplifyChordsMapper.pickle',max_count=100)

In [ ]:
class LstmPredictor(BasePredictor):
    def encode(self, songs):
        self.enc = LabelEncoder()
        self.enc.fit(np.hstack([song[1] for song in songs]))
        
        allX, allY = [], []
        for i, song in enumerate(songs):
            X_notes, X_chords, y = self.encode_song(song)

            allX.append([X_notes, X_chords])
            allY.append(y)
        
        return allX, allY
    
    def encode_song(self, song):
        X_notes = np.array(song[0][:-1])
        X_chords = enc.transform(song[1][:-1])
        y = enc.transform(song[1][1:])
        return X_notes, X_chords, y

    def predict_song(self, song_c, startwith='pause'):
        song = self.ccm.process(song_c)[0]

        X_chords, X_notes, y = self.encode_song(song)
    
        if startwith == 'pause':
            chords = [self.enc.transform([''])[0]]
        elif startwith == 'triad':
            chords = [self.enc.transform([song_c.melody_track.chords[0].notes[0].get_music21_repr().name])[0]]
        
        for note in X_notes:
            predicted_chord_num = merged_model.predict(
                [np.array([chords[-1]]), np.array([note])], 
                batch_size=1).argmax()
            
            chords.append(predicted_chord_num)
        chords = [self.enc.inverse_transform(num) for num in chords]
        merged_model.reset_states()
        
        predicted_song = deepcopy(song_c)
        inversed = self.ccm.inverse_process([[], chords])
        predicted_song.chord_track.chords = inversed.chord_track.chords
        return predicted_song

In [ ]:
pred = LstmPredictor()
allX, allY = pred.encode(songs)
pred.load_dicts(DATASET_HOME+'simple/simple_dataset_dicts.pickle')

In [ ]:
predicted_song = pred.predict_song(corpus.songs[0], startwith='triad')

In [ ]:
predicted_song

In [ ]:
predicted_song.get_music21_repr().show()

In [ ]:
predicted_song.get_music21_repr().show('midi')